# Clases y funciones

## Clases

## Funciones

### Cálculo de distancia cliente y comercio

In [1]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) ** 2 +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Cargar librerias y dataset

## Cargar librerias

In [2]:
import time
import folium
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import concurrent.futures
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


## Cargar dataset

In [3]:
url= "./../data/credit_card_transactions.csv"
data = pd.read_csv(url)
data_copy = data.copy()

# Preprocesamiento

## Análisis exploratorio

### Información general

In [ ]:
data_copy.info()

A partir del metodo .info se puedo encontrar esta información relevante:

* Tiene un total de registro 1.296.675 entradas y 24 columnas en el data
* Tipos datos:int64, float64 y object.
* Valores nulos: Solamente la columna merch_zipcode tiene valores nulos NaN

### Muestra aleatoria

In [ ]:
data_copy.sample(30)

### Cantidad de valores nulos

In [ ]:
data_copy.isnull().sum()

Se confirma la información obtenida con el método .info respecto a la columna merch_zipcode, la cual presenta un total de 195,973 valores nulos. Se recomienda realizar un análisis exhaustivo de la distribución de estos nulos para determinar si están concentrados en ciertas categorías o tipos de transacciones.

Es fundamental evaluar cómo la falta de información en esta columna podría afectar el análisis, especialmente en el contexto de la detección de fraudes. Esta evaluación permitirá comprender el impacto que puede tener la ausencia de datos en la precisión y la efectividad de los modelos de detección.

A partir de este análisis, será necesario decidir el método a utilizar para la imputación de datos o considerar la eliminación de la columna si su relevancia resulta limitada.

### Cantidad de valores unicos

In [ ]:
data_copy.nunique()

La información obtenida de los valores únicos del conjunto de datos es la siguiente:

* **Transacciones y Clientes:** Se registraron un total de 1,296,675 transacciones, con solo 983 clientes únicos. Esto indica que muchos de estos clientes están realizando múltiples transacciones.

* **Diversidad de Comercios:** La columna merchant contiene 693 valores únicos, lo que sugiere una diversidad de comercios o tiendas donde se realizan las transacciones.

* **Clasificación de Categorías:** La columna category tiene solo 14 valores únicos, lo que indica que las transacciones se pueden clasificar en un número limitado de categorías.

* **Inconsistencias Geográficas:** Las columnas city, state, zip, lat, long, y city_pop muestran valores únicos que no coinciden entre sí. Esto podría deberse a:

        * Un cliente puede residir en una ciudad con múltiples códigos postales, o un código postal puede abarcar varias ciudades.

        * Pueden existir errores de entrada que resulten en códigos postales incorrectos o datos de ubicación duplicados.

        * Clientes que utilizan diferentes direcciones o que realizan transacciones en diferentes lugares.

* **Diversidad Demográfica:** Las columnas gender, job, y dob indican una diversidad en la base de clientes.

* **Posibles Duplicados:** Las columnas first y last tienen valores diferentes, lo que podría ser indicativo de clientes que comparten el mismo nombre o apellido, o podría señalar la existencia de valores duplicados.

En base a la información obtenida, se pueden implementar las siguientes estrategias de análisis:

* **Frecuencia de Compras por Cliente:** Explorar la frecuencia de compras por cliente para identificar patrones de comportamiento que puedan indicar riesgo de fraude.

* **Detección de compras varias del mismo cliente:**  Utilizar las columnas trans_num y cc_num para detectar posibles duplicados y errores en los datos.

* **Análisis Geográfico:** Examinar la distribución de las transacciones en función de las ubicaciones (ciudad, estado, etc.) para encontrar patrones que puedan ayudar a identificar fraudes.

* **Análisis Demográfico:** Investigar cómo las características demográficas pueden relacionarse con la probabilidad de fraude.

### Detección de duplicados

#### Duplicados generales

In [ ]:
print("Esta es la cantidad de duplicados: ",data_copy[data_copy.duplicated()]["cc_num"].count())

#### Transacciones múltiples

##### Transacciones múltiples por tarjeta de crédito

In [ ]:
dupli_ccnum = data_copy[data_copy["cc_num"].duplicated()]
print(f"Esta es la cantidad total de transacciones múltiples por tarjeta de crédito: {dupli_ccnum['cc_num'].count()}")

##### Análisis de frecuencia de uso de tarjetas de crédito

In [ ]:
pivot_dupli = dupli_ccnum.pivot_table(values=['trans_num', 'amt'], index='cc_num', aggfunc={'trans_num': 'count', 'amt': 'sum'}).sort_values(by='trans_num')
pivot_dupli.rename(columns={
    'trans_num': 'Cantidad de Usos',
    'amt': 'Suma Total de Transacciones'
}, inplace=True)

pivot_dupli.sort_values(by='Suma Total de Transacciones', ascending=False, inplace=True)

pivot_dupli

In [ ]:
pivot_dupli.describe()

A continuación, se presenta la información relevante extraída del método .describe() en el DataFrame pivot_dupli:

* **Columna cantidad de usos**:
    * El promedio de transacciones por tarjeta es aproximadamente 1,318.01, lo que sugiere un uso frecuente de las tarjetas.
    * La desviación estándar es notablemente alta (62% de la media), lo que indica que algunas tarjetas tienen un número de transacciones significativamente mayor que otras.
    * La tarjeta con el menor número de transacciones fue utilizada 6 veces, mientras que la de mayor uso lo fue 3,122 veces. Esto sugiere que existen usuarios muy activos con sus tarjetas.

* **Columna suma total de transacciones**:
    * El monto promedio de las transacciones es de 92,707.06, lo que indica un uso relativamente alto de las tarjetas en comparación con las transacciones diarias típicas.
    * La desviación estándar también es alta (64% de la media), lo que sugiere una gran variabilidad en los montos, con algunas transacciones significativamente más altas que otras.
    * El monto mínimo de una transacción es de 1,038.43, lo que indica la existencia de transacciones de menor valor, mientras que el monto máximo es de 296,353.63, lo que puede ser indicativo de transacciones inusualmente grandes y potencialmente fraudulentas.

    **La amplia diferencia entre el número mínimo y máximo de transacciones, así como entre los montos mínimos y máximos, resalta la necesidad de realizar un análisis exhaustivo para detectar posibles fraudes. La alta desviación estándar en ambas columnas sugiere la presencia de comportamientos atípicos en el uso de tarjetas de crédito, lo que facilitará el proceso de análisis de fraudes..**

##### Cantidad de transacciones duplicadas

In [ ]:
print("Esta es la cantidad de transacciones cantidad de transacciones duplicadas:",data_copy[data_copy["trans_num"].duplicated()]["trans_num"].count())

### Muestra estadística descriptiva

In [ ]:
data_copy.describe()

La información que se puede extraer del método .describe() es la siguiente:

* La columna cc_num no proporciona información relevante, ya que solo contiene números de tarjeta de crédito. Sin embargo, podemos observar que los números de tarjeta van desde 60412621000 hasta 4992346000000000000.

* El promedio de amt (monto de la transacción) es aproximadamente 70,35. El rango de transacciones está entre 1 y 28.948,90, con una desviación estándar de aproximadamente 160,31. Esto indica una gran variabilidad en los montos de transacción.

* La columna zip varía entre 1257 y 99780, lo que indica que cubre una amplia variedad de áreas geográficas.

* Las columnas lat (latitud) varían entre 20,03° y 66,69°, y long (longitud) entre -165,67° y -67,95°, cubriendo áreas desde Alaska hasta el este de los Estados Unidos.

* En la columna city_pop (población de la ciudad), encontramos ciudades con poblaciones muy pequeñas (23 habitantes) y grandes metrópolis con hasta 2.906.700 habitantes.

* Las columnas merch_lat y merch_long (latitud y longitud del comercio) tienen valores muy similares a las columnas lat y long, lo que indica que los comercios están distribuidos de manera similar a las ubicaciones de los compradores.

* La columna merch_zipcode muestra variaciones significativas, lo que indica que los comercios están geográficamente distribuidos en una amplia variedad de áreas.

**En general, se concluye que existe una gran variabilidad en los montos de las transacciones, aunque la mayoría son relativamente pequeñas (media = 70,35). Además, la latitud, longitud y los códigos postales muestran que el dataset abarca una gran cantidad de regiones geográficas. Las ciudades donde ocurren las transacciones varían enormemente en población, desde ciudades pequeñas (23 habitantes) hasta grandes metrópolis (2.906.700 habitantes).**

### Matriz de correlación

In [ ]:
numeric_columns = data_copy.select_dtypes(include=['number']).columns

correlation_matrix = data_copy[numeric_columns].corr()

plt.figure(figsize=(12, 10))

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm',
            square=True, linewidths=.5, cbar_kws={"shrink": .8})

plt.title('Matriz de Correlación', fontsize=16)
plt.show()

* **Relación entre is_fraud y las variables geográficas (zip, lat, long, merch_lat, merch_zipcode):** Las correlaciones extremadamente bajas (cercanas a 0) indican que la ubicación geográfica (tanto del comprador como del comerciante) no parece tener un impacto significativo en si una transacción es fraudulenta. Este hallazgo sugiere que los fraudes no están concentrados en zonas geográficas específicas.

* **Relación entre amt y is_fraud:** Hay una correlación relativamente alta entre el monto de la transacción (amt) y is_fraud, lo que indica que las transacciones de mayor monto tienen más probabilidad de ser fraudulentas. Esto sugiere que el monto es un factor relevante para detectar fraude.

* **Relación entre Unnamed: 0 y unix_time:** Existe una correlación casi perfecta entre Unnamed: 0 y unix_time. Esto sugiere que Unnamed: 0 es simplemente un índice basado en el tiempo o el orden de las transacciones, lo que lo hace redundante. Podría eliminarse en el análisis.

* **Relación entre zip y merch_zipcode:** La alta correlación entre zip y merch_zipcode indica que muchas transacciones ocurren entre compradores y comerciantes que están geográficamente cercanos. Dado que esta información es redundante, puedes eliminar una de las dos columnas. Preferiblemente, eliminar merch_zipcode, ya que tienes la columna merchant que contiene el nombre del comercio y es más relevante.


**En conclusión:**
* **La variable is_fraud está más correlacionada con el monto de la transacción (amt) que con cualquier otra variable, lo que sugiere que las transacciones de mayor monto tienen mayor probabilidad de ser fraudulentas.**

* **Las variables geográficas no muestran correlaciones significativas con el fraude, lo que implica que el fraude no está ligado a ubicaciones geográficas específicas.**

* **Existen relaciones fuertes entre las ubicaciones de los compradores y comerciantes, lo que indica que las transacciones tienden a ocurrir en áreas geográficamente cercanas.**

* **Es recomendable eliminar columnas con alta correlación (como Unnamed: 0 y merch_zipcode) por su naturaleza redundante, lo que permitirá enfocarse en las variables más relevantes para la predicción de fraude. Además, eliminando las columnas first y last probablamente no son útiles para identificar si una transacción es fraudulenta o no, esta no suele tener un impacto significativo en los patrones de fraude, porque el fraude no está relacionado con el nomnbre del cliente, sino con comportamiento anómalos, montos, numero de transacciones y ubicaciones**

# Procesamiento

## Enriquecimiento de datos

### Creación de nuevas caracteristicas

In [15]:
# Se crea columna edad.
data_copy['dob'] = pd.to_datetime(data_copy['dob'])
data_copy['age'] = np.floor((pd.Timestamp.now() - data_copy['dob']).dt.days / 365.25).astype("int64")

# Se crea columna distancia entre el cliente y comercio
data_copy['distance'] = data_copy.apply(lambda row: haversine(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)

# Se crea columna monto promedio por transacción
data_copy['amount_per_transaction'] = data_copy.groupby('cc_num')['amt'].transform('mean')

# Se crea columna relación entre el monto y la edad del titular
data_copy['amount_per_age'] = data_copy['amt'] / data_copy['age']

# Se crea columna días desde la última transacción
data_copy['trans_date_trans_time'] = pd.to_datetime(data_copy['trans_date_trans_time'])
data_copy['days_since_last_transaction'] = data_copy.groupby('cc_num')['trans_date_trans_time'].diff().dt.days.fillna(0)

# Se crea columna monto total gastado por tarjeta
data_copy['total_spent'] = data_copy.groupby('cc_num')['amt'].transform('sum')

# Se crea columna diferencia de monto respecto al promedio
data_copy['amount_diff'] = data_copy['amt'] - data_copy.groupby('cc_num')['amt'].transform('mean')

# Se crea columna frecuencia de uso
data_copy['transaction_count'] = data_copy.groupby('cc_num')['cc_num'].transform('count')

# Se crea columna relacion entre el monto y la frecuencia de uso
data_copy['amount_frequency_ratio'] = data_copy['amount_per_transaction'] / data_copy['transaction_count']

# Se crea columna diferencia de tiempo entre transacciones
data_copy['time_diff'] = data_copy.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds().fillna(0)

# Se crea columna interacción entre la edad y el monto
data_copy['age_amount_interaction'] = data_copy['age'] * data_copy['amt']

# Se crea columna categorías de edad
bins = [0, 25, 35, 45, 55, 65, 100]
labels = ['<25', '25-34', '35-44', '45-54', '55-64', '65+']
data_copy['age_group'] = pd.cut(data_copy['age'], bins=bins, labels=labels, right=False)

# Se crea columna frecuencia de transacciones por cliente
transaction_counts = data_copy['cc_num'].value_counts()
data_copy['transaction_count'] = data_copy['cc_num'].map(transaction_counts)

# Se crea columna frecuencia de transacciones por comercio
merchant_counts = data_copy['merchant'].value_counts()
data_copy['transaction_count_by_merchant'] = data_copy['merchant'].map(merchant_counts)

# Se crea columna para saber cuantas categorias tiene el comercio
unique_category_counts = data_copy.groupby('merchant')['category'].nunique()
data_copy['nunique_category'] = data_copy['merchant'].map(unique_category_counts)


### Eliminar columnas de dataset

In [16]:
data_copy = data_copy.drop(columns=["dob"])
data_copy = data_copy.drop(columns=["Unnamed: 0", "unix_time","merch_zipcode"])

# Análisis

## Análisis de todas las transacciones

### Distribución General de las Categorías de Comercios

In [ ]:
# Agrupar comercios por categoría
category_counts = data_copy.groupby('category')['merchant'].count().reset_index()


# Ordenar los resultados
category_counts = category_counts.sort_values(by='merchant', ascending=False)

# Graficar la distribución por categorías
fig = px.bar(category_counts, 
              x='category', 
              y='merchant', 
              title='Número de Transacciones por Categoría de Comercio',
              labels={'category': 'Categoría de Comercio', 'merchant': 'Número de Transacciones'})

# Mejorar la visualización
fig.update_layout(xaxis_tickangle=-90)

# Mostrar el gráfico interactivo
fig.show()

### Número de transacciones por comercio en cada categoría

In [ ]:
categories = data_copy['category'].unique()

all_merchant_counts = []

for category in categories:
    category_data = data_copy[data_copy['category'] == category]
    
    merchant_counts = category_data['merchant'].value_counts().reset_index()
    merchant_counts.columns = ['merchant', 'transaction_count']
    
    total_transactions = merchant_counts['transaction_count'].sum()
    
    merchant_counts['percentage'] = (merchant_counts['transaction_count'] / total_transactions) * 100
    
    merchant_counts['category'] = category
    
    all_merchant_counts.append(merchant_counts)

all_merchant_counts_df = pd.concat(all_merchant_counts, ignore_index=True)

fig = px.bar(all_merchant_counts_df, 
              x='merchant', 
              y='transaction_count', 
              color='category',
              title='Número de Transacciones por Comerciante en Cada Categoría',
              labels={'merchant': 'Comerciante', 'transaction_count': 'Número de Transacciones'},
              barmode='stack')

fig.update_layout(xaxis_tickangle=-90)

fig.show()


In [ ]:
result = all_merchant_counts_df.groupby('merchant').agg(
    transaction_count=('transaction_count', 'sum'),
    percentage=('percentage', 'sum'),
    category=('category', lambda x: ', '.join(set(x)))
).reset_index()

result = result.sort_values(by='transaction_count', ascending=False).head(10)

result

### Comercios con múltiples categorías

In [ ]:
merchant_categories = data_copy.groupby('merchant')['category'].agg(lambda x: list(set(x))).reset_index()

merchant_categories.columns = ['merchant', 'categories']

multiple_categories_merchants = merchant_categories[merchant_categories['categories'].apply(lambda x: len(x) > 1)]

multiple_categories_merchants = multiple_categories_merchants.set_index('merchant')

multiple_categories_merchants

### Transacciones con montos más frecuentes

In [ ]:
frequent_amounts = data_copy['amt'].value_counts().reset_index()
frequent_amounts.columns = ['amount', 'frequency']

frequent_amounts = frequent_amounts.sort_values(by='frequency', ascending=False)

bins = [0, 100, 500, 1000, 5000, 10000, 30000] 
labels = ['0-100', '101-500', '501-1000', '1001-5000', '5001-10000', '10001-30000']

frequent_amounts['amount_range'] = pd.cut(frequent_amounts['amount'], bins=bins, labels=labels, right=False)

grouped_data = frequent_amounts.groupby('amount_range')['frequency'].sum().reset_index()

fig = px.bar(
    grouped_data,
    x='amount_range',
    y='frequency',
    title='Frecuencia de los Montos por Rangos',
    labels={'amount_range': 'Rango de Montos', 'frequency': 'Frecuencia'},
    text='frequency' 
)

fig.update_layout(xaxis_title='Rango de Montos', yaxis_title='Frecuencia', bargap=0.2)

fig.show()


In [ ]:
fig = px.bar(
    frequent_amounts.head(100),
    x='amount',
    y='frequency',
    title='Top 100 Montos Más Frecuentes en Transacciones',
    labels={'amount': 'Montos', 'frequency': 'Frecuencia'},
    color='frequency',
    color_continuous_scale='Blues'
)

fig.show()

### Monto total por comerciante

In [ ]:
merchant_volumes = data_copy.groupby('merchant')['amt'].sum().reset_index()
merchant_volumes.columns = ['merchant', 'total_volume']

total_volume = merchant_volumes['total_volume'].sum()

merchant_volumes['percentage_of_total'] = (merchant_volumes['total_volume'] / total_volume) * 100

merchant_volumes = merchant_volumes.sort_values(by='total_volume', ascending=False)

merchant_volumes.head(10) 


### Análisis descriptivo

#### Visualizacion

In [ ]:
fig1 = px.box(data_copy, y='amt', title='Boxplot de Monto (amt) para Todas las Transacciones')
fig1.update_yaxes(title_text='Monto (amt)')
fig1.show()

# Crear un boxplot para la edad
fig2 = px.box(data_copy, y='age', title='Boxplot de Edad para Todas las Transacciones')
fig2.update_yaxes(title_text='Edad')
fig2.show()

# Crear un boxplot para la distancia
fig3 = px.box(data_copy, y='distance', title='Boxplot de Distancia para Todas las Transacciones')
fig3.update_yaxes(title_text='Distancia')
fig3.show()

In [ ]:
# Crea un objeto de figura y una cuadrícula de subgráficas
fig, axs = plt.subplots(3, 1, figsize=(12, 18))

# Histograma para 'amt'
sns.histplot(data_copy['amt'], bins=30, kde=True, ax=axs[0])
axs[0].set_title('Distribución del Monto (amt) para Transacciones')
axs[0].set_xlabel('Monto (amt)')
axs[0].set_ylabel('Frecuencia')

# Histograma para 'age'
sns.histplot(data_copy['age'], bins=30, kde=True, ax=axs[1])
axs[1].set_title('Distribución de Edad para Transacciones')
axs[1].set_xlabel('Edad')
axs[1].set_ylabel('Frecuencia')

# Histograma para 'distance'
sns.histplot(data_copy['distance'], bins=30, kde=True, ax=axs[2])
axs[2].set_title('Distribución de Distancia para Transacciones')
axs[2].set_xlabel('Distancia')
axs[2].set_ylabel('Frecuencia')

# Ajustar el espacio entre subgráficas
plt.tight_layout()
plt.show()

#### Correlaciones

In [ ]:
numeric_columns = data_copy.select_dtypes(include='number')

correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(20, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación para Transacciones')
plt.show()

#### Segmentación de tiempo

##### Segmentación de tiempo día de la semana

In [ ]:
data_copy['day_of_week'] = data_copy['trans_date_trans_time'].dt.day_name()

weekly_count = data_copy.groupby('day_of_week').size()

days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_count = weekly_count.reindex(days_order, fill_value=0)

fig = go.Figure(data=[
    go.Bar(
        x=weekly_count.index,  
        y=weekly_count.values,  
        marker_color='orange'  
    )
])

fig.update_layout(
    title="Conteo de Transacciones por Día de la Semana",
    xaxis_title="Día de la Semana",
    yaxis_title="Número de Transacciones",
    xaxis_tickangle=-45,  
    width=800,
    height=500
)

fig.show()

##### Segmentación tiempo por semana

In [ ]:
weekly_count = data_copy.groupby(data_copy['trans_date_trans_time'].dt.isocalendar().week).size().reset_index(name='count')

weekly_count.columns = ['Semana del Año', 'Número de Transacciones']

fig = px.bar(weekly_count, 
             x='Semana del Año', 
             y='Número de Transacciones', 
             title='Conteo de Transacciones por Semana',
             labels={'Semana del Año': 'Semana del Año', 'Número de Transacciones': 'Número de Transacciones'},
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500)  

fig.show()

##### Segmentación tiempo mensual

In [ ]:
monthly_count = data_copy.groupby(data_copy['trans_date_trans_time'].dt.month).size().reset_index(name='count')

monthly_count.columns = ['Mes', 'Número de Transacciones']

fig = px.bar(monthly_count, 
             x='Mes', 
             y='Número de Transacciones', 
             title='Conteo de Transacciones por Mes',
             labels={'Mes': 'Mes', 'Número de Transacciones': 'Número de Transacciones'},
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  


fig.update_traces(texttemplate='%{text:.2s}', textposition='outside') 
fig.update_layout(xaxis=dict(tickmode='linear'), 
                  height=500, 
                  xaxis_title='Mes',  
                  yaxis_title='Número de Transacciones')  


fig.show()


##### Segmentación de tiempo trimestral

In [ ]:
quarterly_count = data_copy.groupby(data_copy['trans_date_trans_time'].dt.to_period('Q')).size().reset_index(name='count')

quarterly_count['Trimestre'] = quarterly_count['trans_date_trans_time'].dt.strftime('%Y-Q%q')

quarterly_count = quarterly_count[['Trimestre', 'count']]

quarterly_count.columns = ['Trimestre', 'Número de Transacciones']

fig = px.bar(quarterly_count, 
             x='Trimestre', 
             y='Número de Transacciones',  
             title='Conteo de Transacciones por Trimestre',  
             labels={'Trimestre': 'Trimestre', 'Número de Transacciones': 'Número de Transacciones'},  
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500,  
                  xaxis_title='Trimestre',  
                  yaxis_title='Número de Transacciones')  


fig.show()


##### Segmentación de tiempo semestral

In [ ]:
data_copy['semester'] = (data_copy['trans_date_trans_time'].dt.month - 1) // 6 + 1

semiannual_count = data_copy.groupby('semester').size().reset_index(name='count')

fig = px.bar(semiannual_count, 
             x='semester', 
             y='count', 
             title='Conteo de Transacciones por Semestre', 
             labels={'semester': 'Semestre', 'count': 'Número de Transacciones'}, 
             text='count',  
             color='count',  
             color_continuous_scale=px.colors.sequential.Viridis)  

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500,  
                  xaxis_title='Semestre', 
                  yaxis_title='Número de Transacciones') 

fig.show()

##### Segmentación de tiempo anual

In [ ]:
yearly_count = data_copy.groupby(data_copy['trans_date_trans_time'].dt.year).size().reset_index(name='count')

# Crear la gráfica de barras interactiva
fig = px.bar(yearly_count, 
             x='trans_date_trans_time',  # Eje x como el año
             y='count', 
             title='Conteo de Transacciones por Año', 
             labels={'trans_date_trans_time': 'Año', 'count': 'Número de Transacciones'}, 
             text='count',  # Mostrar el conteo en las barras
             color='count',  # Color según el número de transacciones
             color_continuous_scale=px.colors.sequential.Viridis)  # Escala de color

# Actualizar el diseño
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  # Formato de texto
fig.update_layout(xaxis=dict(tickmode='linear'),  # Modo de ticks del eje x
                  height=500,  # Altura de la figura
                  xaxis_title='Año',  # Título del eje x
                  yaxis_title='Número de Transacciones')  # Título del eje y

# Mostrar el gráfico interactivo
fig.show()

#### Análisis de variables cuantitativas

##### Análisis de distancia

In [ ]:
plt.figure(figsize=(10, 5))
# Crear el gráfico de dispersión y la línea de regresión
sns.regplot(x='distance', y='amt', data=data_copy, scatter_kws={'alpha':0.5}, line_kws={"color":"red"})
plt.title('Relación entre Distancia y Monto de Transacciones Fraudulentas')
plt.xlabel('Distancia')
plt.ylabel('Monto')
plt.show()

In [ ]:
# Crear el pairplot con hue para distinguir entre fraude y no fraude
sns.pairplot(data_copy[['amount_per_transaction', 'amount_per_age', 
                        'amount_frequency_ratio', 'age_amount_interaction', 'amount_diff', 'is_fraud']], 
             diag_kind='kde', corner=True, hue='is_fraud', palette='coolwarm')

# Añadir título
plt.suptitle('Relaciones entre las Variables Segmentadas por Fraude', y=1.02)

# Mostrar el gráfico
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Distribuciones por target usando hue para `is_fraud`
sns.boxplot(data=data_copy, x='is_fraud', y='amount_per_transaction', ax=axes[0, 0])
axes[0, 0].set_title('amount_per_transaction vs is_fraud')

sns.boxplot(data=data_copy, x='is_fraud', y='amount_per_age', ax=axes[0, 1])
axes[0, 1].set_title('amount_per_age vs is_fraud')

sns.boxplot(data=data_copy, x='is_fraud', y='amount_frequency_ratio', ax=axes[0, 2])
axes[0, 2].set_title('amount_frequency_ratio vs is_fraud')

sns.boxplot(data=data_copy, x='is_fraud', y='age_amount_interaction', ax=axes[1, 0])
axes[1, 0].set_title('age_amount_interaction vs is_fraud')

sns.boxplot(data=data_copy, x='is_fraud', y='amount_diff', ax=axes[1, 1])
axes[1, 1].set_title('amount_diff vs is_fraud')

axes[1, 2].axis('off')  # Ocultar último eje

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# KDE plots para comparar distribuciones
sns.kdeplot(data=data_copy, x='amount_per_transaction', hue='is_fraud', fill=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribución de amount_per_transaction')

sns.kdeplot(data=data_copy, x='amount_per_age', hue='is_fraud', fill=True, ax=axes[0, 1])
axes[0, 1].set_title('Distribución de amount_per_age')

sns.kdeplot(data=data_copy, x='amount_frequency_ratio', hue='is_fraud', fill=True, ax=axes[0, 2])
axes[0, 2].set_title('Distribución de amount_frequency_ratio')

sns.kdeplot(data=data_copy, x='age_amount_interaction', hue='is_fraud', fill=True, ax=axes[1, 0])
axes[1, 0].set_title('Distribución de age_amount_interaction')

sns.kdeplot(data=data_copy, x='amount_diff', hue='is_fraud', fill=True, ax=axes[1, 1])
axes[1, 1].set_title('Distribución de amount_diff')

axes[1, 2].axis('off')  # Ocultar último eje

plt.tight_layout()
plt.show()




In [ ]:
from scipy.stats import ttest_ind

# Realizar t-tests para cada variable
variables = ['amount_per_transaction', 'amount_per_age', 
             'amount_frequency_ratio', 'age_amount_interaction', 'amount_diff']

for var in variables:
    fraud = data_copy[data_copy['is_fraud'] == 1][var]
    non_fraud = data_copy[data_copy['is_fraud'] == 0][var]
    
    stat, p = ttest_ind(fraud, non_fraud, equal_var=False)
    print(f"T-test para {var}: estadístico={stat:.4f}, p-valor={p:.4f}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Definir variables predictoras y target
X = data_copy[['amount_per_transaction', 'amount_per_age', 
               'amount_frequency_ratio', 'age_amount_interaction', 'amount_diff']]
y = data_copy['is_fraud']

# Separar en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar modelo de regresión logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Mostrar los coeficientes del modelo
coef_df = pd.DataFrame(model.coef_[0], index=X.columns, columns=['Coeficiente'])
print(coef_df.sort_values(by='Coeficiente', ascending=False))


#### Análsis de variable categóricas

In [ ]:
categorical_vars = ['category', 'gender', 'state','age_group']

plt.figure(figsize=(15, len(categorical_vars) * 5))
for i, var in enumerate(categorical_vars):
    plt.subplot(len(categorical_vars), 1, i + 1)
    sns.countplot(x=var, hue='is_fraud', data=data_copy)
    plt.title(f'Distribución de {var} para Fraude')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
amount_per_transaction, amount_per_age, amount_frequency_ratio, age_amount_interaction,amount_diff

## Análisis Transacciones Fraudulentas

In [34]:
fraudulent_transactions = data_copy[data_copy["is_fraud"]==1]

### Análisis descriptivo

#### Visualización

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.boxplot(y='amt', data=fraudulent_transactions, ax=axes[0])
axes[0].set_title('Boxplot de Monto (amt) para Transacciones Fraudulentas')
axes[0].set_ylabel('Monto (amt)')

sns.boxplot(y='age', data=fraudulent_transactions, ax=axes[1])
axes[1].set_title('Boxplot de Edad para Transacciones Fraudulentas')
axes[1].set_ylabel('Edad')

sns.boxplot(y='distance', data=fraudulent_transactions, ax=axes[2])
axes[2].set_title('Boxplot de Distancia para Transacciones Fraudulentas')
axes[2].set_ylabel('Distancia')

plt.tight_layout()
plt.show()

**Boxplot amt:**

Las transacciones fraudulentas tienden a involucrar montos relativamente pequeños, con una mediana que sugiere que la mayoría de los fraudes ocurren en cantidades más manejables. Sin embargo, existen algunas transacciones significativamente grandes que distorsionan la distribución, lo que indica que los fraudes de alto valor, aunque menos comunes, representan un riesgo considerable.

**Boxplot edad:**

La mayoría de las transacciones fraudulentas involucran a personas de edad media (entre 38 y 65 años), lo que sugiere que este grupo podría estar más expuesto o involucrado en fraudes. Sin embargo, los valores atípicos en edades muy jóvenes o mayores pueden indicar excepciones notables, lo que sugiere la necesidad de prestar atención a fraudes en estos extremos de edad.

**Boxplot distancia:**

Las transacciones fraudulentas tienden a ocurrir a distancias moderadas del usuario (entre 58 y 100 unidades), lo que sugiere que los fraudes a menudo se realizan en ubicaciones cercanas. Sin embargo, los valores atípicos en los extremos, tanto en distancias muy cortas como muy largas, podrían indicar fraudes inusuales, con posibles implicaciones de acceso cercano o actividades fraudulentas a gran distancia.

In [ ]:
# Crea un objeto de figura y una cuadrícula de subgráficas
fig, axs = plt.subplots(3, 1, figsize=(12, 18))

# Histograma para 'amt'
sns.histplot(fraudulent_transactions['amt'], bins=30, kde=True, ax=axs[0])
axs[0].set_title('Distribución del Monto (amt) para Transacciones Fraudulentas')
axs[0].set_xlabel('Monto (amt)')
axs[0].set_ylabel('Frecuencia')

# Histograma para 'age'
sns.histplot(fraudulent_transactions['age'], bins=30, kde=True, ax=axs[1])
axs[1].set_title('Distribución de Edad para Transacciones Fraudulentas')
axs[1].set_xlabel('Edad')
axs[1].set_ylabel('Frecuencia')

# Histograma para 'distance'
sns.histplot(fraudulent_transactions['distance'], bins=30, kde=True, ax=axs[2])
axs[2].set_title('Distribución de Distancia para Transacciones Fraudulentas')
axs[2].set_xlabel('Distancia')
axs[2].set_ylabel('Frecuencia')

# Ajustar el espacio entre subgráficas
plt.tight_layout()
plt.show()

**Histograma amt:**

La mayoría de los fraudes involucran montos relativamente pequeños, pero los montos altos, aunque poco frecuentes, podrían representar operaciones fraudulentas más dañinas. Este comportamiento es típico en fraudes donde los delincuentes intentan evadir sospechas con transacciones pequeñas para no ser detectados los cuales son los oportunistas, pero ocasionalmente prueban realizar fraudes con montos más altos, los cuales suelen ser más planeados y dañinos debido a su monton elevado.

**Histograma edades:**

Las transacciones fraudulentas parecen afectar principalmente a personas de mediana edad, lo cual puede estar relacionado con su mayor actividad económica o su vulnerabilidad a ciertos tipos de fraude por el uso de tecnologías. Sin embargo, no se deben ignorar los fraudes en personas jóvenes o mayores, ya que, aunque menos frecuentes, también ocurren

**Histograma distancia:**

Las transacciones fraudulentas tienden a ocurrir cerca de la ubicación del usuario, lo que sugiere que los delincuentes pueden tener acceso cercano al usuario o estar usando métodos para disfrazar su ubicación. Los fraudes lejanos, aunque menos comunes,  podrían involucrar estafadores que operan desde ubicaciones remotas.

#### Correlaciones

In [ ]:
numeric_columns = fraudulent_transactions.select_dtypes(include='number')

correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(20, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación para Transacciones Fraudulentas')
plt.show()

Las correlaciones observadas en el mapa de calor son en general débiles, lo que indica que dentro del subconjunto de transacciones fraudulentas no existen relaciones claras entre las variables principales como el monto de la transacción, la edad del titular y la distancia. Esto sugiere que estos factores, en este análisis específico, no son indicadores determinantes de fraude. Sin embargo, sería valioso explorar otras variables del dataset, como la categoría del comercio o los patrones temporales, que podrían ofrecer una mejor capacidad predictiva.

La relación fuerte entre el monto (amt) y las columnas derivadas como amount_per_age, amount_diff y age_amount_interaction revela que los usuarios de mayor edad tienden a realizar transacciones de montos más altos. Esto podría deberse a su mayor actividad económica, lo que los hace un blanco atractivo para fraudes de montos elevados. Además, la diferencia entre montos en transacciones sucesivas podría ser un claro indicador de fraude, ya que cambios bruscos pueden levantar sospechas. La interacción entre edad y monto también sugiere que los estafadores podrían apuntar a usuarios mayores con transacciones de mayor valor.

En cuanto a la relación entre amount_per_transaction y las columnas fraud_ratio y amount_frecuency_ratio, se observa que las transacciones frecuentes con montos altos tienen una mayor probabilidad de ser fraudulentas. Esto es coherente con la idea de que los delincuentes podrían aprovechar montos elevados para maximizar el impacto del fraude. Por otro lado, la correlación negativa con total_spent y transaction_count indica que las cuentas con un historial de gasto más limitado son más propensas a intentos de fraude en transacciones grandes, posiblemente para evitar ser detectados en cuentas menos activas.

Finalmente, la relación entre time_diff y days_since_last_transaction muestra que, a mayor tiempo entre transacciones, mayor es la diferencia de tiempo registrada entre ellas. Este patrón puede ser útil para identificar comportamientos sospechosos, como intervalos largos de inactividad seguidos de una transacción fraudulenta, lo que sugiere una táctica común entre los estafadores para aprovechar momentos de baja actividad en las cuentas.



In [ ]:
# Pairplot para las variables seleccionadas
sns.pairplot(fraudulent_transactions[['amt', 'age', 'distance', 'total_spent']])
plt.suptitle('Pairplot de Transacciones Fraudulentas', y=1.02)
plt.show()

El análisis del pairplot revela información clave sobre las transacciones fraudulentas y cómo interactúan entre las variables.

En primer lugar, observamos que, en cuanto a las distribuciones individuales, los datos muestran que la mayoría de las transacciones fraudulentas se concentran en montos relativamente bajos. Sin embargo, es importante señalar que los fraudes de montos más altos, aunque menos frecuentes, son particularmente significativos y representan un mayor riesgo debido al valor que involucran. Con respecto a la edad, las personas de mediana edad parecen ser las más afectadas por las transacciones fraudulentas, lo cual puede estar relacionado con su mayor actividad económica o su vulnerabilidad a ciertos tipos de estafa. En cuanto a la distancia, los fraudes tienden a ocurrir cerca de la ubicación del usuario, lo que sugiere un acceso físico cercano por parte del estafador o el uso de técnicas para ocultar la ubicación real del atacante.

Al analizar las relaciones entre pares de variables, se observa que el monto y la distancia presentan una tendencia interesante: los fraudes de mayor monto suelen ocurrir en proximidad al domicilio del usuario, lo que puede ser indicativo de que el delincuente tiene acceso físico o cercano al titular de la cuenta. En cuanto a la relación entre monto y edad, los datos sugieren que las personas mayores tienden a estar asociadas con fraudes de mayor monto, lo que puede indicar que los estafadores se dirigen a este grupo debido a que manejan más dinero o son percibidos como más vulnerables. Por otro lado, la relación entre edad y distancia no muestra una correlación significativa, lo que sugiere que la proximidad física del fraude no está directamente relacionada con la edad del titular de la cuenta. Los estafadores parecen actuar tanto en fraudes cercanos como lejanos sin una preferencia clara por la edad del usuario.

En un análisis general de fraude y variables, se identifica que los fraudes tienden a concentrarse en montos bajos y a ocurrir cerca del domicilio del usuario, mientras que los montos elevados, aunque menos frecuentes, podrían estar relacionados con estafas mejor planificadas. Las transacciones no fraudulentas, por su parte, están distribuidas de manera más amplia en todas las variables, lo que resalta una clara diferencia en los patrones de comportamiento entre las transacciones fraudulentas y no fraudulentas. Estos patrones permiten perfilar mejor las características comunes de los fraudes.

Finalmente, al considerar los valores atípicos (outliers), observamos que hay transacciones fraudulentas con montos muy altos o distancias inusualmente grandes o pequeñas. Estos casos atípicos podrían representar fraudes más complejos o sofisticados, que no siguen el comportamiento general de los fraudes más comunes. Estos outliers son clave para identificar posibles ajustes en los modelos de detección de fraude, ya que podrían evadir los sistemas de detección actuales debido a sus características inusuales.

#### Segmentación de tiempo

##### Segmentación tiempo por día

In [ ]:
fraudulent_transactions['day_of_week'] = fraudulent_transactions['trans_date_trans_time'].dt.day_name()

weekly_fraud_count = fraudulent_transactions.groupby('day_of_week').size()

days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_count = weekly_fraud_count.reindex(days_order, fill_value=0)

fig = go.Figure(data=[
    go.Bar(
        x=weekly_count.index,  
        y=weekly_count.values,  
        marker_color='orange'  
    )
])

fig.update_layout(
    title="Conteo de Transacciones Fraudulentas por Día de la Semana",
    xaxis_title="Día de la Semana",
    yaxis_title="Número de Transacciones Fraudulentas",
    xaxis_tickangle=-45,  
    width=800,
    height=500
)

fig.show()

##### Segmentación tiempo por semana

In [ ]:
weekly_count = fraudulent_transactions.groupby(fraudulent_transactions['trans_date_trans_time'].dt.isocalendar().week).size().reset_index(name='count')

# Renombrar las columnas
weekly_count.columns = ['Semana del Año', 'Número de Transacciones']

# Crear el gráfico de barras
fig = px.bar(weekly_count, 
             x='Semana del Año', 
             y='Número de Transacciones', 
             title='Conteo de Transacciones Fraudulentas por Semana',
             labels={'Semana del Año': 'Semana del Año', 'Número de Transacciones': 'Número de Transacciones'},
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  

# Actualizar las trazas del gráfico
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500)  

# Mostrar el gráfico
fig.show()

In [ ]:
weekly_fraud_count = fraudulent_transactions.groupby(data_copy['trans_date_trans_time'].dt.isocalendar().week).size()
plt.figure(figsize=(20, 5))
weekly_fraud_count.plot(kind='bar')
plt.title('Conteo de Transacciones Fraudulentas por Semana')
plt.xlabel('Semana del Año')
plt.ylabel('Número de Transacciones Fraudulentas')
plt.xticks(rotation=0)
plt.show()


##### Segmentación tiempo mensual

In [ ]:
monthly_count = fraudulent_transactions.groupby(fraudulent_transactions['trans_date_trans_time'].dt.month).size().reset_index(name='count')

monthly_count.columns = ['Mes', 'Número de Transacciones']

fig = px.bar(monthly_count, 
             x='Mes', 
             y='Número de Transacciones', 
             title='Conteo de Transacciones Fraudulentas por Mes',
             labels={'Mes': 'Mes', 'Número de Transacciones': 'Número de Transacciones'},
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside') 
fig.update_layout(xaxis=dict(tickmode='linear'), 
                  height=500, 
                  xaxis_title='Mes',  
                  yaxis_title='Número de Transacciones')  

fig.show()

##### Segmentación tiempo trimestral

In [ ]:
quarterly_count = fraudulent_transactions.groupby(fraudulent_transactions['trans_date_trans_time'].dt.to_period('Q')).size().reset_index(name='count')

quarterly_count['Trimestre'] = quarterly_count['trans_date_trans_time'].dt.strftime('%Y-Q%q')

quarterly_count = quarterly_count[['Trimestre', 'count']]

quarterly_count.columns = ['Trimestre', 'Número de Transacciones']

fig = px.bar(quarterly_count, 
             x='Trimestre', 
             y='Número de Transacciones',  
             title='Conteo de Transacciones Fraudulentas por Trimestre',  
             labels={'Trimestre': 'Trimestre', 'Número de Transacciones': 'Número de Transacciones'},  
             text='Número de Transacciones',  
             color='Número de Transacciones',  
             color_continuous_scale=px.colors.sequential.Viridis)  

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500,  
                  xaxis_title='Trimestre',  
                  yaxis_title='Número de Transacciones')  

fig.show()

##### Segmentación tiempo semestral

In [ ]:
fraudulent_transactions['semester'] = (fraudulent_transactions['trans_date_trans_time'].dt.month - 1) // 6 + 1

semiannual_count = fraudulent_transactions.groupby('semester').size().reset_index(name='count')

fig = px.bar(semiannual_count, 
             x='semester', 
             y='count', 
             title='Conteo de Transacciones Fraudulentas por Semestre', 
             labels={'semester': 'Semestre', 'count': 'Número de Transacciones'}, 
             text='count',  
             color='count',  
             color_continuous_scale=px.colors.sequential.Viridis)  

# Actualizar el diseño
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500,  
                  xaxis_title='Semestre', 
                  yaxis_title='Número de Transacciones') 

# Mostrar el gráfico
fig.show()



##### Segmentación tiempo anual

In [ ]:
yearly_count = fraudulent_transactions.groupby(fraudulent_transactions['trans_date_trans_time'].dt.year).size().reset_index(name='count')

fig = px.bar(yearly_count, 
             x='trans_date_trans_time',  
             y='count', 
             title='Conteo de Transacciones Fraudulentas por Año', 
             labels={'trans_date_trans_time': 'Año', 'count': 'Número de Transacciones'}, 
             text='count',  
             color='count',  
             color_continuous_scale=px.colors.sequential.Viridis)  


fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  
fig.update_layout(xaxis=dict(tickmode='linear'),  
                  height=500,  
                  xaxis_title='Año',  
                  yaxis_title='Número de Transacciones')  

fig.show()


#### Análisis de variables cuantitativas

##### Análisis de distancia

In [ ]:
plt.figure(figsize=(10, 5))
# Crear el gráfico de dispersión y la línea de regresión
sns.regplot(x='distance', y='amt', data=fraudulent_transactions, scatter_kws={'alpha':0.5}, line_kws={"color":"red"})
plt.title('Relación entre Distancia y Monto de Transacciones Fraudulentas')
plt.xlabel('Distancia')
plt.ylabel('Monto')
plt.show()

Se observa "bandas horizontales", lo que indica que los montos de las transacciones fraudulentas tienden a repetirse a ciertos niveles. Estos patrones sugieren que los fraudes con **montos bajos** o **moderados** (por ejemplo, alrededor de $200 y $600) son recurrentes. Esto podría deberse a que los estafadores buscan pasar desapercibidos evitando transacciones de gran valor, ya que estas suelen activar alertas automáticas de los sistemas de detección de fraude. La alta densidad de puntos para montos bajos podría indicar que los fraudes en pequeñas cantidades son muy comunes. Los criminales pueden preferir realizar muchas transacciones pequeñas en lugar de pocas de gran monto, ya que estas últimas suelen generar más atención y posiblemente disparen mecanismos de control.

Aunque la mayoría de las transacciones fraudulentas están agrupadas en rangos más bajos, se puede observar que hay algunas transacciones fraudulentas de montos elevados (alrededor de los $1000), pero estas son menos frecuentes. Los fraudes de alto valor tienden a ser más raros, probablemente porque los sistemas de detección de fraude y las medidas de seguridad de las tarjetas de crédito son más estrictos cuando se trata de transacciones de montos elevados. Los estafadores podrían evitar grandes cantidades debido al riesgo de detección.

En cuanto a la distancia, los fraudes parecen ocurrir a lo largo de un amplio rango (de 0 hasta 140 unidades de distancia). La gran dispersión de puntos indica que los fraudes no están limitados a distancias cortas o largas. Transacciones que ocurren a largas distancias (mayores de 100) podrían implicar situaciones en las que la ubicación del comercio es significativamente diferente de la del titular de la tarjeta, lo cual es un comportamiento típico en fraudes. Sin embargo, la presencia de fraudes en distancias cortas podría indicar casos de fraude interno o uso de tarjetas robadas en proximidad al titular.

Observando la regresión nos podría sugerir que la distancia no es un factor determinante en el monto de las transacciones fraudulentas. Los estafadores parecen operar sin importar la distancia entre el lugar de la transacción y el titular de la tarjeta. Las transacciones fraudulentas podrían estar distribuidas de manera uniforme a lo largo de las distancias, lo que apunta a una globalización del fraude, donde las transacciones pueden ocurrir tanto localmente como en lugares lejanos con igual probabilidad.

#### Análisis de variables categóricas

##### Distribución de variables ['category', 'gender', 'state', 'age_group']

In [ ]:
categorical_vars = ['category', 'gender', 'state','age_group']

plt.figure(figsize=(15, len(categorical_vars) * 5))
for i, var in enumerate(categorical_vars):
    plt.subplot(len(categorical_vars), 1, i + 1)
    sns.countplot(x=var, hue='is_fraud', data=fraudulent_transactions)
    plt.title(f'Distribución de {var} para Fraude')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

##### Distribución de variable ["job"]

In [26]:
fraud_counts_job = fraudulent_transactions['job'].value_counts().reset_index()
fraud_counts_job.columns = ['job', 'fraud_count']

bins = [0, 10, 20, 30, 40, 50, 60, fraud_counts_job['fraud_count'].max() + 1]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60+']
fraud_counts_job['range'] = pd.cut(fraud_counts_job['fraud_count'], bins=bins, labels=labels, right=False)

fraud_counts_job['group'] = fraud_counts_job['range'].cat.codes + 1

table = fraud_counts_job[['group', 'job', 'fraud_count']]

grouped_sums = fraud_counts_job.groupby('group')['fraud_count'].sum().reset_index()
grouped_sums.columns = ['group', 'total_fraud_count']

table = table.merge(grouped_sums, on='group', how='left')

final_table = table.loc[table.groupby('group')['fraud_count'].idxmax()]

final_table.set_index('group', inplace=True)

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 5))

# Total Fraud Count vs Group
plt.subplot(1, 2, 1)
sns.barplot(x=final_table.index, y='total_fraud_count', data=final_table, palette='viridis')
plt.title('Total Fraud Count vs Group job')
plt.xlabel('Group Job')
plt.ylabel('Total Fraud Count')

# Fraud Count vs Group
plt.subplot(1, 2, 2)
sns.barplot(x=final_table.index, y='fraud_count', data=final_table, palette='viridis')
plt.title('Fraud Count vs Group Job')
plt.xlabel('Group Job')
plt.ylabel('Fraud Count')

plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

filtered_groups = [4, 5, 6, 7]
filtered_table = table[table['group'].isin(filtered_groups)]

groups = filtered_table['group'].unique()

for group in groups:
    plt.figure(figsize=(10, 6))
    group_data = filtered_table[filtered_table['group'] == group]

    sns.barplot(x='job', y='fraud_count', data=group_data, palette='viridis')

    plt.title(f'Fraud Count vs Job for Group {group}')
    plt.xlabel('Job')
    plt.ylabel('Fraud Count')
    plt.xticks(rotation=45, ha='right')

    plt.tight_layout()
    plt.show()

In [ ]:
filtered_table = table[table['group'].isin([1, 2, 3])]

for group in range(1, 4):  
    print(f"\nGroup {group} Data:")
    group_data = filtered_table[filtered_table['group'] == group]
    print(group_data[['job', 'fraud_count']])
    group_data.describe()

    print("\nStatistics for 'fraud_count':")
    print(group_data['fraud_count'].describe())

##### Tasa de Fraude Comercio vs Dia de semana

In [ ]:
# Crear la tabla dinámica para calcular la proporción de fraudes por categoría y día de la semana
combined_analysis = data_copy.pivot_table(values='is_fraud',
                                          index='category',
                                          columns='day_of_week',
                                          aggfunc='mean')

# Visualizar la tasa de fraude combinada
plt.figure(figsize=(12, 8))
sns.heatmap(combined_analysis, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Tasa de Fraude por Categoría de Comercio y Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Categoría de Comercio')
plt.show()


Las categorías de compras en línea (**shopping_net y misc_net**) presentan consistentemente tasas de fraude más altas que las categorías físicas, lo que sugiere que las transacciones online siguen siendo el objetivo principal de los defraudadores. Especialmente, el **viernes** es el día con más actividad fraudulenta en estas categorías, lo que podría relacionarse con un aumento en las compras previas al fin de semana o con un mayor descuido por parte de los consumidores.

Las transacciones en puntos de venta físicos de supermercados (**grocery_pos**)  también tienen tasas de fraude elevadas. Esto puede deberse a la frecuencia con la que se usan estos puntos para transacciones diarias, lo que les hace un objetivo atractivo para los fraudes. La tasa más alta se observa el jueves, lo que podría estar relacionado con un aumento en las compras pre-fin de semana.

En varias categorías, los **jueves y viernes** parecen ser los días de mayor riesgo de fraude. Estos días pueden ser especialmente críticos para las medidas de seguridad en las plataformas de comercio electrónico y las tiendas físicas.

Los datos revelan que las **transacciones en línea** y **puntos de venta físicos de supermercados** son más vulnerables al fraude, especialmente los jueves y viernes. Las empresas en estas categorías podrían beneficiarse al mejorar sus sistemas de seguridad durante estos días, cuando el fraude es más prevalente. Una atención especial a estas tendencias puede ayudar a mitigar el riesgo y proteger mejor tanto a los consumidores como a las empresas.



##### Distribución de variable ["merchant"]

In [31]:
fraud_counts_merchant = fraudulent_transactions['merchant'].value_counts().reset_index()
fraud_counts_merchant.columns = ['merchant', 'fraud_count']

bins = [0, 10, 20, 30, 40, fraud_counts_merchant['fraud_count'].max() + 1]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50']
fraud_counts_merchant['range'] = pd.cut(fraud_counts_merchant['fraud_count'], bins=bins, labels=labels, right=False)


fraud_counts_merchant['group'] = fraud_counts_merchant['range'].cat.codes + 1

table_merchant = fraud_counts_merchant[['group', 'merchant', 'fraud_count']]

grouped_sums_merchant = fraud_counts_merchant.groupby('group')['fraud_count'].sum().reset_index()
grouped_sums_merchant.columns = ['group', 'total_fraud_count']

table_merchant = table_merchant.merge(grouped_sums, on='group', how='left')

final_table_merchant = table_merchant.loc[table_merchant.groupby('group')['fraud_count'].idxmax()]

final_table_merchant.set_index('group', inplace=True)

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 5))

# Total Fraud Count vs Group
plt.subplot(1, 2, 1)
sns.barplot(x=final_table_merchant.index, y='total_fraud_count', data=final_table_merchant, palette='viridis')
plt.title('Total Fraud Count vs Group Merchant')
plt.xlabel('Group Merchant')
plt.ylabel('Total Fraud Count')

# Fraud Count vs Group
plt.subplot(1, 2, 2)
sns.barplot(x=final_table_merchant.index, y='fraud_count', data=final_table_merchant, palette='viridis')
plt.title('Fraud Count vs Group Merchant')
plt.xlabel('Group Merchant')
plt.ylabel('Fraud Count')

plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

filtered_groups = [5]
filtered_table_merchant = table_merchant[table_merchant['group'].isin(filtered_groups)]

groups = filtered_table_merchant['group'].unique()

for group in groups:
    plt.figure(figsize=(10, 6))
    group_data_merchant = filtered_table_merchant[filtered_table_merchant['group'] == group]

    sns.barplot(x='merchant', y='fraud_count', data=group_data_merchant, palette='viridis')

    plt.title(f'Fraud Count vs Merchant for Group {group}')
    plt.xlabel('Merchant')
    plt.ylabel('Fraud Count')
    plt.xticks(rotation=45, ha='right')

    plt.tight_layout()
    plt.show()

In [ ]:
filtered_table_merchant = table_merchant[table_merchant['group'].isin([1, 2, 3, 4])]

for group in range(1, 5):
    print(f"\nGroup {group} Data:")
    group_data = filtered_table_merchant[filtered_table_merchant['group'] == group]
    print(group_data[['merchant', 'fraud_count']])
    group_data.describe()

    print("\nStatistics for 'fraud_count':")
    print(group_data['fraud_count'].describe())

In [ ]:
sns.histplot(data_copy['amt'])
plt.show()


In [ ]:
fraud_count = data_copy['is_fraud'].value_counts()
px.bar(fraud_count, title="Distribución de Fraude")


In [ ]:
plt.figure(figsize=(15, 8))
sns.violinplot(x='category', y='amt', data=data_copy, palette='viridis')
plt.title('Distribución del Monto por Categoría')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(x='category', y='amt', data=data_copy, palette='coolwarm')
plt.title('Distribución del Monto por Categoría')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
g = sns.FacetGrid(data_copy, col='category', col_wrap=4, height=4, sharex=False, sharey=False)
g.map(sns.histplot, 'amt', bins=50)
g.set_titles('{col_name}')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=data_copy, x='amt', hue='category', bins=50, multiple='stack')
plt.title('Histograma de Montos por Categoría')
plt.show()


In [ ]:
plt.figure(figsize=(15, 8))
sns.scatterplot(x='trans_date_trans_time', y='amt', hue='category', data=data_copy, alpha=0.6)
plt.title('Monto de Transacción por Categoría en el Tiempo')
plt.xticks(rotation=45)
plt.show()

### ESTO SE HACE CON TODO EL DATA SET COMPLETO

In [ ]:
# Análisis comparativo de tasa de fraude por categoría y comerciante en todo el dataset

# Agrupar por categoría y comerciante
category_fraud_analysis = data_copy.groupby(['category', 'merchant']).agg(
    total_transactions=('is_fraud', 'size'),
    total_frauds=('is_fraud', 'sum')
).reset_index()

# Calcular tasa de fraude por categoría y comerciante
category_fraud_analysis['fraud_rate'] = category_fraud_analysis['total_frauds'] / category_fraud_analysis['total_transactions']

# Gráfico de la tasa de fraude por categoría y comerciante
plt.figure(figsize=(12, 6))
sns.barplot(data=category_fraud_analysis, x='category', y='fraud_rate', hue='merchant', palette='viridis')
plt.title('Tasa de Fraude por Categoría y Comerciante (Todo el dataset)')
plt.xticks(rotation=90)
plt.ylabel('Tasa de Fraude')
plt.xlabel('Categoría')
plt.legend(title='Comerciante', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()  # Ajustar el diseño
plt.show()



### ESTE CODIGO SE HACE CON EL data_copy completo, quiere decir con is_fraud= 0 y is_fraud=1

In [ ]:
# Asegúrate de que la columna trans_date_trans_time sea del tipo datetime
fraudulent_transactions['trans_date_trans_time'] = pd.to_datetime(fraudulent_transactions['trans_date_trans_time'])

# Extraer el día de la semana y la hora
fraudulent_transactions['day_of_week'] = fraudulent_transactions['trans_date_trans_time'].dt.day_name()
fraudulent_transactions['hour'] = fraudulent_transactions['trans_date_trans_time'].dt.hour

# Agrupar por día de la semana y calcular la tasa de fraude
time_analysis = fraudulent_transactions.groupby('day_of_week').agg({
    'is_fraud': 'mean',  # Tasa de fraude
    'amt': 'mean'  # Monto promedio
}).reset_index()

# Visualizar la tasa de fraude por día de la semana
plt.figure(figsize=(12, 6))
sns.barplot(x='day_of_week', y='is_fraud', data=time_analysis.sort_values('is_fraud', ascending=False))
plt.title('Tasa de Fraude por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Tasa de Fraude')
plt.xticks(rotation=45)
plt.show()


### ESTE CODIGO SE HACE CON EL data_copy completo, quiere decir con is_fraud= 0 y is_fraud=1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Filtra el DataFrame para incluir solo transacciones fraudulentas
fraud_data = data_copy[data_copy['is_fraud'] == 1]

# Cuenta el número de fraudes por ocupación
fraud_by_job = fraud_data['job'].value_counts().reset_index()
fraud_by_job.columns = ['job', 'fraud_count']

# Agrupa los trabajos por rango
bins = [0, 10, 20, 30, 40, 50, fraud_by_job['fraud_count'].max()]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60']
fraud_by_job['range'] = pd.cut(fraud_by_job['fraud_count'], bins=bins, labels=labels, right=False)

# Cuenta el número de trabajos en cada rango
fraud_range_counts = fraud_by_job['range'].value_counts().sort_index()

# Gráfico general de fraudes por rango de ocupación
plt.figure(figsize=(10, 6))
plt.bar(fraud_range_counts.index.astype(str), fraud_range_counts.values, color='skyblue')
plt.ylabel('Número de Trabajos')
plt.title('Número de Trabajos por Rango de Fraudes')
plt.xticks(rotation=45)
plt.show()

### ESTO SE HACE CON EL CODIGO COMPLETO

In [ ]:
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Filtrar los datos para transacciones fraudulentas y no fraudulentas
fraudulent_data = data_copy[data_copy['is_fraud'] == 1]
non_fraudulent_data = data_copy[data_copy['is_fraud'] == 0]

# Z-score para detectar outliers en ambas categorías
fraudulent_z_scores = stats.zscore(fraudulent_data[['amt', 'distance']])
non_fraudulent_z_scores = stats.zscore(non_fraudulent_data[['amt', 'distance']])

# Identificar outliers
fraudulent_outliers = (abs(fraudulent_z_scores) > 3).any(axis=1)
non_fraudulent_outliers = (abs(non_fraudulent_z_scores) > 3).any(axis=1)

# Visualizar outliers
plt.figure(figsize=(15, 10))

# Boxplot para montos de transacciones fraudulentas
plt.subplot(2, 2, 1)
sns.boxplot(x=fraudulent_data['amt'])
plt.title('Boxplot de Montos de Transacciones Fraudulentas')

# Boxplot para montos de transacciones no fraudulentas
plt.subplot(2, 2, 2)
sns.boxplot(x=non_fraudulent_data['amt'])
plt.title('Boxplot de Montos de Transacciones No Fraudulentas')

# Boxplot para distancias de transacciones fraudulentas
plt.subplot(2, 2, 3)
sns.boxplot(x=fraudulent_data['distance'])
plt.title('Boxplot de Distancias de Transacciones Fraudulentas')

# Boxplot para distancias de transacciones no fraudulentas
plt.subplot(2, 2, 4)
sns.boxplot(x=non_fraudulent_data['distance'])
plt.title('Boxplot de Distancias de Transacciones No Fraudulentas')

plt.tight_layout()
plt.show()


In [ ]:
# Z-score para detectar outliers
z_scores = stats.zscore(data_copy[['amt', 'distance']])
outliers = (abs(z_scores) > 3).any(axis=1)

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
sns.boxplot(x=data_copy['amt'])
plt.title('Boxplot de Montos de Transacciones')

plt.subplot(1, 2, 2)
sns.boxplot(x=data_copy['distance'])
plt.title('Boxplot de Distancias de Transacciones')

plt.tight_layout()
plt.show()

### ESTE CODIGO SE HACE CON EL data_copy completo, quiere decir con is_fraud= 0 y is_fraud=1

### Análisis por Categorias del Comercio PERO CON TODO EL DATASET data_copy

In [ ]:
# Agrupar por categoría y calcular estadísticas descriptivas
category_analysis = data_copy.groupby('category').agg({
    'amt': ['mean', 'median', 'count'],  # Estadísticas sobre el monto
    'is_fraud': 'mean'  # Proporción de fraudes por categoría
}).reset_index()

# Renombrar columnas para facilitar la interpretación
category_analysis.columns = ['category', 'avg_amt', 'median_amt', 'transaction_count', 'fraud_rate']

# Visualizar la tasa de fraude por categoría
plt.figure(figsize=(12, 6))
sns.barplot(x='fraud_rate', y='category', data=category_analysis.sort_values('fraud_rate', ascending=False))
plt.title('Tasa de Fraude por Categoría de Comercio')
plt.xlabel('Tasa de Fraude')
plt.ylabel('Categoría')
plt.show()



In [ ]:
category_analysis

Interpretación de los Resultados
Montos Promedio y Mediana:

grocery_pos tiene el monto promedio más alto (116.96) y una mediana considerable (105.12). Esto podría indicar que las compras en tiendas de comestibles físicas tienden a ser más altas en comparación con otras categorías.
shopping_net también muestra un monto promedio alto (88.42) pero con una mediana baja (8.44), lo que sugiere que hay algunas compras de gran valor que están afectando el promedio, mientras que muchas transacciones son de bajo monto.
Conteo de Transacciones:

gas_transport y shopping_pos tienen un alto número de transacciones (131,659 y 116,672 respectivamente), lo que podría indicar que son categorías muy populares. Sin embargo, el hecho de que shopping_net tenga una tasa de fraude (0.0176) más alta que otras categorías podría ser un punto de interés para investigar más a fondo.
Tasa de Fraude:

shopping_net tiene la tasa de fraude más alta (1.76%), seguida de misc_net (1.45%) y grocery_pos (1.41%). Esto indica que estas categorías son más propensas a transacciones fraudulentas en comparación con otras.
La tasa de fraude para food_dining (0.17%) y health_fitness (0.15%) es considerablemente baja, lo que podría sugerir que estas categorías son más seguras.
Recomendaciones para el Análisis Adicional
Investigación en Profundidad de Categorías de Alto Fraude:

Considera hacer un análisis más detallado sobre las transacciones en las categorías de shopping_net, misc_net, y grocery_pos. Podrías investigar factores como:
¿Qué tipo de transacciones dentro de estas categorías son más propensas al fraude?
¿Existen características específicas de los clientes o patrones de comportamiento que se asocian con fraudes en estas categorías?


Análisis Temporal:

Podrías analizar cómo varía la tasa de fraude en diferentes categorías a lo largo del tiempo. Esto puede ayudarte a identificar si hay tendencias estacionales o patrones específicos de tiempo asociados con el fraude.
Comparaciones de Montos:

Considera comparar los montos promedio y mediana entre categorías de fraude y no fraude para obtener más información sobre cómo el monto de la transacción puede influir en la probabilidad de fraude.
Relación con Otras Variables:

Examina cómo se relacionan otras variables, como la edad del titular de la tarjeta y la distancia a la que se realiza la transacción, con las categorías y la tasa de fraude.
Conclusión
Tu análisis proporciona una base sólida para explorar patrones de fraude en tu conjunto de datos. Al profundizar en categorías específicas y sus características, puedes identificar tendencias y potencialmente desarrollar modelos más precisos para detectar fraudes.

## Análisis Transacciones No Fraudulentas

In [ ]:
non_fraudulent_transactions = data_copy[data_copy['is_fraud'] == 0]
non_fraudulent_transactions.describe()

### Cálculo de distancias Cliente y Comercio

In [ ]:
# Dividir el DataFrame en transacciones fraudulentas y no fraudulentas
fraudulent_transactions = data_copy[data_copy['is_fraud'] == 1]
non_fraudulent_transactions = data_copy[data_copy['is_fraud'] == 0]

# Visualizar la distribución de las distancias
plt.figure(figsize=(12, 6))
sns.histplot(fraudulent_transactions['distance'], bins=50, color='red', label='Fraudulentas', kde=True)
sns.histplot(non_fraudulent_transactions['distance'], bins=50, color='blue', label='No Fraudulentas', kde=True)
plt.show()



In [ ]:
# Verificar si hay valores nulos
print(data_copy['distance'].isnull().sum())  # Muestra el número de valores nulos


In [ ]:
# Calcular estadísticas descriptivas de la columna de distancia
stats = data_copy['distance'].describe(percentiles=[0.25, 0.5, 0.75, 0.90, 0.95])
stats


In [ ]:
# Definir el umbral de distancia como el percentil 90
threshold_distance = stats['90%']  # o stats['95%'] si prefieres

# Crear un DataFrame combinado para el gráfico
combined_data = pd.DataFrame({
    'Distance': pd.concat([fraudulent_transactions['distance'], non_fraudulent_transactions['distance']]),
    'Type': ['Fraudulent'] * len(fraudulent_transactions) + ['Non Fraudulent'] * len(non_fraudulent_transactions)
})

# Crear un histograma interactivo con Plotly
fig = px.histogram(combined_data, x='Distance', color='Type', barmode='overlay',
                   labels={'Distance': 'Distancia (km)', 'Type': 'Tipo de Transacción'},
                   title='Distribución de Distancias para Transacciones Fraudulentas y No Fraudulentas',
                   color_discrete_map={'Fraudulent': 'red', 'Non Fraudulent': 'blue'})  # Colores deseados

# Agregar línea del umbral
fig.add_shape(type="line", x0=threshold_distance, x1=threshold_distance, y0=0, y1=1000,  # Ajusta y1 según la escala del gráfico
              line=dict(color="green", dash="dash"), name='Umbral de Distancia')

# Personalizar el layout
fig.update_layout(yaxis_title='Frecuencia', xaxis_title='Distancia (km)')
fig.show()



In [ ]:
# Definir el umbral de distancia como el percentil 90
threshold_distance = stats['90%']  # o stats['95%'] si prefieres

# Visualizar la distribución de las distancias
plt.figure(figsize=(12, 6))

# Graficar las transacciones fraudulentas y no fraudulentas
sns.histplot(fraudulent_transactions['distance'], bins=50, color='red', label='Fraudulentas', kde=True)
sns.histplot(non_fraudulent_transactions['distance'], bins=50, color='blue', label='No Fraudulentas', kde=True)

# Agregar línea del umbral
plt.axvline(x=threshold_distance, color='green', linestyle='--', label='Umbral de Distancia')
plt.title('Distribución de Distancias para Transacciones Fraudulentas y No Fraudulentas')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()


In [ ]:
# Definir el umbral de distancia (usando el percentil 90)
threshold_distance = data_copy['distance'].describe(percentiles=[0.90])['90%']

# Marcar las transacciones potencialmente fraudulentas
data_copy['potential_fraud'] = data_copy['distance'] > threshold_distance

# Crear el gráfico interactivo
fig = px.histogram(data_copy, x='distance', color='potential_fraud',
                   color_discrete_map={True: 'red', False: 'blue'},
                   labels={'potential_fraud': 'Fraude Potencial'},
                   title='Distribución de Distancias para Transacciones',
                   nbins=50)

# Agregar línea del umbral
fig.add_shape(type='line',
              x0=threshold_distance, y0=0,
              x1=threshold_distance, y1=data_copy['distance'].max(),
              line=dict(color='green', width=2, dash='dash'))

# Actualizar el layout
fig.update_layout(xaxis_title='Distancia (km)',
                  yaxis_title='Frecuencia',
                  showlegend=True)

# Mostrar el gráfico
fig.show()

In [ ]:


# Crear un gráfico de barras para visualizar la relación entre la categoría y el fraude
plt.figure(figsize=(12, 6))
sns.countplot(data=data_copy, x='category', hue='is_fraud', palette='Set1')
plt.title('Número de Transacciones por Categoría y Clase de Fraude')
plt.xlabel('Categoría')
plt.ylabel('Número de Transacciones')
plt.legend(title='Fraude', labels=['No', 'Sí'])
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=data_copy, x='is_fraud', y='amt', palette='Set1')
plt.title('Distribución del Monto de Transacciones según Fraude')
plt.xlabel('Clase de Fraude')
plt.ylabel('Monto de Transacción')
plt.xticks(ticks=[0, 1], labels=['No', 'Sí'])
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=data_copy, x='gender', hue='is_fraud', palette='Set1')
plt.title('Número de Transacciones por Género y Clase de Fraude')
plt.xlabel('Género')
plt.ylabel('Número de Transacciones')
plt.legend(title='Fraude', labels=['No', 'Sí'])
plt.show()


In [ ]:
plt.figure(figsize=(14, 7))
sns.countplot(data=data_copy, x='state', hue='is_fraud', palette='Set1')
plt.title('Número de Transacciones por Estado y Clase de Fraude')
plt.xlabel('Estado')
plt.ylabel('Número de Transacciones')
plt.legend(title='Fraude', labels=['No', 'Sí'])
plt.xticks(rotation=45)
plt.show()


In [ ]:
data_copy.nunique()

In [ ]:
# 1. Análisis Descriptivo
fraud_counts = data_copy['is_fraud'].value_counts()
print(fraud_counts)

# Visualizar la distribución de is_fraud
sns.countplot(data=data_copy, x='is_fraud')
plt.title('Distribución de Transacciones: Fraudulentas vs Legítimas')
plt.show()

In [ ]:
sns.boxplot(x='is_fraud', y='amt', data=data_copy)
plt.title('Distribución de Monto por Clase de Fraude')
plt.show()

In [ ]:
# 3. Gráfico de barras por categoría
fraud_by_category = data_copy.groupby(['category', 'is_fraud']).size().unstack()
fraud_by_category.plot(kind='bar', stacked=True)
plt.title('Transacciones Fraudulentas y Legítimas por Categoría')
plt.ylabel('Número de Transacciones')
plt.show()

In [ ]:
# 4. Correlación de variables numéricas
# Seleccionar solo columnas numéricas
numeric_df = data_copy.select_dtypes(include=['float64', 'int64'])

# Calcular la matriz de correlación
correlation_matrix = numeric_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# 5. Gráfico de dispersión
sns.scatterplot(data=data_copy, x='distance', y='amt', hue='is_fraud', alpha=0.5)
plt.title('Monto vs Distancia según Clase de Fraude')
plt.show()

In [ ]:
data_copy[data_copy['is_fraud'] == 0].groupby(by="age_group").agg("count")

In [ ]:
# 6. Análisis por Edad

# 6.1. Crear histograma separado para transacciones fraudulentas y legítimas
plt.figure(figsize=(12, 6))
sns.histplot(data=data_copy[data_copy['is_fraud'] == 0], x='age', color='blue', bins=30, label='No Fraud', alpha=0.3, stat='density')
sns.histplot(data=data_copy[data_copy['is_fraud'] == 1], x='age', color='red', bins=30, label='Fraud', alpha=0.6, stat='density')
plt.title('Distribución de Edad según Clase de Fraude (Histograma Separado)')
plt.xlabel('Edad')
plt.ylabel('Densidad')
plt.legend()
plt.show()

# 6.2. Gráfico de barras por grupo de edad
# Crear rangos de edad
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
data_copy['age_group'] = pd.cut(data_copy['age'], bins=bins, labels=labels)

# Contar transacciones fraudulentas y no fraudulentas por grupo de edad
age_counts = data_copy.groupby(['age_group', 'is_fraud']).size().unstack().fillna(0)

# Crear gráfico de barras
age_counts.plot(kind='bar', stacked=True)
plt.title('Transacciones por Grupo de Edad y Clase de Fraude')
plt.xlabel('Grupo de Edad')
plt.ylabel('Número de Transacciones')
plt.xticks(rotation=45)
plt.show()

# 6.3. Gráfico de densidad (KDE) separado
plt.figure(figsize=(12, 6))
sns.kdeplot(data=data_copy[data_copy['is_fraud'] == 0], x='age', fill=True, color='blue', label='No Fraud', alpha=0.6)
sns.kdeplot(data=data_copy[data_copy['is_fraud'] == 1], x='age', fill=True, color='red', label='Fraud', alpha=0.6)
plt.title('Distribución de Edad según Clase de Fraude (Densidad Separada)')
plt.xlabel('Edad')
plt.ylabel('Densidad')
plt.legend()
plt.show()

In [ ]:
plt.axvline(x=threshold_distance, color='green', linestyle='--', label='Umbral de Distancia')
plt.title('Distribución de Distancias para Transacciones Fraudulentas y No Fraudulentas')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()

### Análisis Geográfico

In [79]:
fraudulent_transactions = data_copy[data_copy['is_fraud'] == 1]
normal_transactions = data_copy[data_copy['is_fraud'] == 0]

#### Mapa Interactivo transacciones

In [80]:
# Se Crea mapa centrado en las coordenadas promedio
#m_combined = folium.Map(location=[data_copy['lat'].mean(), data_copy['long'].mean()], zoom_start=6)

# Se crea objeto MarkerCluster para las transacciones
#marker_cluster = MarkerCluster().add_to(m_combined)

# Se agrega marcadores para transacciones normales
#for idx, row in normal_transactions.iterrows():
#    folium.Marker(
#        location=[row['lat'], row['long']],
#        popup=f"Transacción: {row['trans_num']}\nMonto: ${row['amt']:.2f}",
#        icon=folium.Icon(color='blue', icon='info-sign')
#    ).add_to(marker_cluster)

# Se agrega marcadores para transacciones fraudulentas
#for idx, row in fraudulent_transactions.iterrows():
#    folium.Marker(
#        location=[row['lat'], row['long']],
#        popup=f"Transacción Fraudulenta: {row['trans_num']}\nMonto: ${row['amt']:.2f}",
#        icon=folium.Icon(color='red', icon='exclamation-sign')
#    ).add_to(marker_cluster)

# Se guarda mapa combinado en un archivo HTML
#map_file = 'mapa_combined_transacciones.html'
#m_combined.save(map_file)
#print(f"Mapa guardado como '{map_file}'")

#### Mapa de Calor de Transacciones Fraudulentas

In [ ]:
# Crear los datos de calor (latitud, longitud)
heat_data = [[row['lat'], row['long']] for _, row in fraudulent_transactions.iterrows()]

# Crear el mapa centrado en las coordenadas promedio
heat_map = folium.Map(location=[fraudulent_transactions['lat'].mean(), fraudulent_transactions['long'].mean()], zoom_start=6)

# Agregar el mapa de calor con radio ajustado
HeatMap(heat_data, radius=15).add_to(heat_map)

# Mostrar el mapa en la celda del Jupyter Notebook en VSC
display(heat_map)  # Este método muestra el mapa interactivo en la celda

# Guardar el mapa en un archivo HTML
map_file = 'mapa_calor_fraud.html'
heat_map.save(map_file)
print(f"Mapa de calor guardado como '{map_file}'")

#### Distribución de Transacciones por Ciudad y Estado

In [82]:
fraud_by_city = fraudulent_transactions.groupby('city').size().reset_index(name='count')
fraud_by_state = fraudulent_transactions.groupby('state').size().reset_index(name='count')

##### Top 10 con más Transacciones Fraudulentas

In [ ]:
top_10_cities = fraud_by_city.sort_values(by='count', ascending=False).head(10)

plt.figure(figsize=(10, 5))
top_10_cities.plot(kind='bar', x='city', y='count', legend=False, ax=plt.gca())
plt.title('Top 10 Ciudades con Más Transacciones Fraudulentas')
plt.ylabel('Número de Transacciones Fraudulentas')
plt.xlabel('Ciudad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


bottom_10_cities = fraud_by_city.sort_values(by='count', ascending=True).head(10)

plt.figure(figsize=(10, 5))
bottom_10_cities.plot(kind='bar', x='city', y='count', legend=False, ax=plt.gca(), color='green')
plt.title('Top 10 Ciudades con Menos Transacciones Fraudulentas')
plt.ylabel('Número de Transacciones Fraudulentas')
plt.xlabel('Ciudad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##### Top 10 con menos Transacciones Fraudulentas

In [ ]:

top_10_states = fraud_by_state.sort_values(by='count', ascending=False).head(10)

plt.figure(figsize=(10, 5))
top_10_states.plot(kind='bar', x='state', y='count', legend=False, ax=plt.gca())
plt.title('Top 10 Estados con Más Transacciones Fraudulentas')
plt.ylabel('Número de Transacciones Fraudulentas')
plt.xlabel('Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

bottom_10_states = fraud_by_state.sort_values(by='count', ascending=True).head(10)

plt.figure(figsize=(10, 5))
bottom_10_states.plot(kind='bar', x='state', y='count', legend=False, ax=plt.gca(), color='green')
plt.title('Top 10 Estados con Menos Transacciones Fraudulentas')
plt.ylabel('Número de Transacciones Fraudulentas')
plt.xlabel('Estado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Frecuencia de Compras por Cliente

In [ ]:
# Agrupar por el número de tarjeta y contar la cantidad de transacciones
compras_por_cliente = data_copy.groupby('cc_num').size().reset_index(name='frecuencia')

# Histograma de frecuencia de compras por cliente
compras_por_cliente['frecuencia'].plot(kind='hist', bins=30)
plt.title('Distribución de la Frecuencia de Compras por Cliente')
plt.xlabel('Número de Compras')
plt.ylabel('Frecuencia')
plt.show()


### Análisis Demográfico

#### Relación entre Edad y Fraude

In [ ]:
# Histograma de edad en transacciones fraudulentas
data_copy[data_copy['is_fraud'] == 1]['age'].plot(kind='hist', bins=30)
plt.title('Distribución de Edad en Transacciones Fraudulentas')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')
plt.show()

#### Relación entre Género y Fraude

In [ ]:
# Conteo de fraudes por género
fraudes_por_genero = data_copy[data_copy['is_fraud'] == 1].groupby('gender').size()

# Graficar la relación entre género y fraude
fraudes_por_genero.plot(kind='bar', color='red')
plt.title('Número de Fraudes por Género')
plt.ylabel('Cantidad de Fraudes')
plt.xlabel('Género')
plt.show()

## Relación de Fraudes y número de transacciones

In [88]:
trans_fraud = data_copy.groupby(by="cc_num").agg(
    total_transc=("trans_num", "count"),
    fraud=("is_fraud", "sum"),
    total_amt = ("amt", "sum")
).reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_transc', y='fraud', data=trans_fraud, label='Datos')
sns.regplot(x='total_transc', y='fraud', data=trans_fraud, scatter=False, color='red', label='Regresión')
plt.title('Relación entre Número de Transacciones y Fraude con Ajuste de Regresión')
plt.xlabel('Número de Transacciones')
plt.ylabel('Número de Fraudes')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_amt', y='fraud', data=trans_fraud, label='Datos')
sns.regplot(x='total_amt', y='fraud', data=trans_fraud, scatter=False, color='red', label='Regresión')
plt.title('Relación entre monto de transacción (amt) y Fraude con Ajuste de Regresión')
plt.xlabel("Monto de transacción")
plt.ylabel('Número de Fraudes')
plt.legend()
plt.show()

In [ ]:
subset = data_copy[['amt', 'lat', 'long', 'age', 'is_fraud']]

sns.pairplot(subset, hue='is_fraud', diag_kind='kde', palette='coolwarm')

plt.savefig('pairplot_fraud_analysis.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import folium
from folium.plugins import FastMarkerCluster
import pandas as pd


# Crea el mapa centrado en una ubicación inicial (puedes ajustar estas coordenadas)
Mm = folium.Map(location=[20.0, 0.0], zoom_start=2)

# Generar una lista de coordenadas para las transacciones
locations = list(zip(df['merch_lat'], df['merch_long']))

# Agregar el FastMarkerCluster al mapa
FastMarkerCluster(locations).add_to(m)

# Guardar el mapa en un archivo HTML
m.save('mapa_fast_marker_cluster.html')

# Mostrar el mapa (si usas Jupyter Notebook, puedes usar esto, pero como estás en VSC, revisa el HTML)
m

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame se llama df y ya contiene la columna unix_time
# Convertir unix_time a datetime
data_copy['trans_date_trans_times'] = pd.to_datetime(data['unix_time'], unit='s')

# Ahora puedes usar la nueva columna trans_date_trans_time para análisis


In [ ]:
data_copy

In [ ]:
data_copy = data.copy()

if data_copy.index.is_monotonic_increasing:
    print("¿El índice es monotónico creciente?", data_copy.index.is_monotonic_increasing)
else:
    print("¿El índice es monotónico decreciente?", data_copy.index.is_monotonic_decreasing)

In [ ]:
data_copy.info()

In [ ]:
# Convertir la columna de fecha a tipo datetime
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

# Agrupar por fecha y contar el número de fraudes
fraudes_por_fecha = data[data['is_fraud'] == 1].groupby(data['trans_date_trans_time'].dt.date).size()

# Visualizar la distribución de fraudes a lo largo del tiempo
plt.figure(figsize=(12, 6))
plt.plot(fraudes_por_fecha.index, fraudes_por_fecha.values, marker='o')
plt.title('Distribución de Fraudes por Fecha')
plt.xlabel('Fecha')
plt.ylabel('Número de Fraudes')
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
fraudes_por_fecha

In [ ]:
# Autocorrelación con diferentes retrasos de tiempo
autocorrelation_lag1 = fraudes_por_fecha.autocorr(lag=1)  # Día
print("One Day Lag: ", autocorrelation_lag1)

autocorrelation_lag7 = fraudes_por_fecha.autocorr(lag=7)  # Semana
print("\nSeven Day Lag: ", autocorrelation_lag7)

autocorrelation_lag30 = fraudes_por_fecha.autocorr(lag=30)  # Mes (aproximadamente 30 días)
print("\nThirty Days Lag: ", autocorrelation_lag30)

autocorrelation_lag90 = fraudes_por_fecha.autocorr(lag=90)  # Trimestre (90 días)
print("\nNinety Days Lag: ", autocorrelation_lag90)

autocorrelation_lag180 = fraudes_por_fecha.autocorr(lag=180)  # Semestre (180 días)
print("\nSix Months Lag: ", autocorrelation_lag180)

autocorrelation_lag365 = fraudes_por_fecha.autocorr(lag=365)  # Año (365 días)
print("\nOne Year Lag: ", autocorrelation_lag365)


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_acf(fraudes_por_fecha, lags=7)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf

# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_pacf(fraudes_por_fecha, lags=7)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_acf(fraudes_por_fecha, lags=30)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_pacf(fraudes_por_fecha, lags=30)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_acf(fraudes_por_fecha, lags=90)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_pacf(fraudes_por_fecha, lags=90)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_acf(fraudes_por_fecha, lags=180)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_pacf(fraudes_por_fecha, lags=180)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_acf(fraudes_por_fecha, lags=365)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
# Visualizar la autocorrelación
plt.figure(figsize=(10, 6))
plot_pacf(fraudes_por_fecha, lags=365)  # Puedes ajustar el número de 'lags'
plt.title('Autocorrelación de Fraudes por Fecha')
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

# Descomponer la serie (ajustar el periodo si es necesario)
decomposition = seasonal_decompose(fraudes_por_fecha, model='additive', period=7)

# Graficar los componentes
decomposition.plot()
plt.show()


In [ ]:
# Descomponer la serie (ajustar el periodo si es necesario)
decomposition = seasonal_decompose(fraudes_por_fecha, model='additive', period=30)

# Graficar los componentes
decomposition.plot()
plt.show()


In [ ]:
# Descomponer la serie (ajustar el periodo si es necesario)
decomposition = seasonal_decompose(fraudes_por_fecha, model='additive', period=90)

# Graficar los componentes
decomposition.plot()
plt.show()

In [ ]:
# Descomponer la serie (ajustar el periodo si es necesario)
decomposition = seasonal_decompose(fraudes_por_fecha, model='additive', period=180)

# Graficar los componentes
decomposition.plot()
plt.show()

In [ ]:
# Descomponer la serie (ajustar el periodo si es necesario)
decomposition = seasonal_decompose(fraudes_por_fecha, model='additive', period=252)

# Graficar los componentes
decomposition.plot()
plt.show()

In [ ]:
fraudes_por_fecha_copy = fraudes_por_fecha.resample("1D").sum()
fraudes_por_fecha_copy_filtered = fraudes_por_fecha["2019-01-06 00:00:00":"2019-01-12 00:00:00"]
fig = px.line(fraudes_por_fecha_copy_filtered, x=fraudes_por_fecha_copy_filtered.index, y="num_orders", title="Fraudes por 7 días")
fig.show()

In [ ]:
fraudes_por_fecha

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='fraud', y='total_transc', data=trans_fraud)
plt.title('Distribución de Fraudes por Número de Transacciones')
plt.xlabel('Fraude')
plt.ylabel('Número de Transacciones')
plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(data[data['is_fraud'] == 1]['amt'], label='Fraude', fill=True)
sns.kdeplot(data[data['is_fraud'] == 0]['amt'], label='No Fraude', fill=True)
plt.title('Densidad de Montos de Fraude vs No Fraude')
plt.xlabel('Monto de la Transacción')
plt.ylabel('Densidad')
plt.legend()
plt.show()


In [ ]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

# Extraer la hora y contar los fraudes por hora
data['hora'] = data['trans_date_trans_time'].dt.hour
fraudes_por_hora = data[data['is_fraud'] == 1].groupby('hora').size()

# Visualizar
plt.figure(figsize=(10, 6))
sns.barplot(x=fraudes_por_hora.index, y=fraudes_por_hora.values)
plt.title('Distribución de Fraudes por Hora')
plt.xlabel('Hora del Día')
plt.ylabel('Número de Fraudes')
plt.xticks(fraudes_por_hora.index)  # Asegúrate de que las horas estén en el eje x
plt.show()


In [ ]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])

# Histograma de tiempos de transacción
plt.figure(figsize=(10, 6))
sns.histplot(data['trans_date_trans_time'].dt.hour, bins=24, kde=True)
plt.title('Distribución de Tiempos de Transacción')
plt.xlabel('Hora del Día')
plt.ylabel('Frecuencia')
plt.show()



In [ ]:
# Calcular IQR para la columna 'amt' (monto de la transacción)
Q1 = data['amt'].quantile(0.25)
Q3 = data['amt'].quantile(0.75)
IQR = Q3 - Q1

# Identificar outliers
outliers = data[(data['amt'] < (Q1 - 1.5 * IQR)) | (data['amt'] > (Q3 + 1.5 * IQR))]

# Visualizar outliers
plt.figure(figsize=(10, 6))
sns.boxplot(x=data['amt'])
plt.title('Identificación de Outliers en Transacciones')
plt.show()

# Mostrar los outliers
print(outliers)
